In [7]:
import csv
import json

In [8]:
import csv
import json

def load_csv(path):
    with open(path,'r') as csvfile:
    
        dict_data = []
        
        reader = csv.reader(csvfile, delimiter=',')
    
        fields = next(reader)
        
        for row in reader:
            dict_row = {}

            for i,f in enumerate(fields):
                value = row[i]
                try:
                    value = int(row[i])
                except ValueError:
                    try:
                        value = float(value)
                    except ValueError:
                        pass
                dict_row[f] = value
                
            dict_data.append(dict_row)
            
    return dict_data



def save_csv(dict_data, path, filename):
    
    with open(f"{path}/{filename}", mode='w', newline='') as file_csv:
        
        columns = dict_data[0].keys()
        
        writer = csv.DictWriter(file_csv, fieldnames=columns)
        writer.writeheader()

        writer.writerows(dict_data)

    print(f"Les données ont été enregistrées dans {path}.")



def save_json(dict_data, path, filename):
    
    with open(f"{path}/{filename}", 'w') as file_json:
        json.dump(dict_data, file_json, indent=2)

    print(f"Les données ont été enregistrées dans {path}.")


def load_json(path):
    with open(path, 'r') as file_json:
        data = json.load(file_json)
        
    return data



def str_type(chaine):
    if chaine.isdigit():
        return "int"
    try:
        float_value = float(chaine)
        return "float" if '.' in chaine else "int"
    except ValueError:
        return "str"
        


def get_file_type(file_path):
    return file_path.split(".")[-1]


def sum_ord(string):
    return sum(ord(c) for c in string)


def get_column_types(data_list):
    column_types = {}

    for item in data_list:
        for key, value in item.items():
            current_type = type(value)

            if key not in column_types:
                column_types[key] = {current_type}
            else:
                column_types[key].add(current_type)

    return column_types

In [149]:
import csv
import json
import copy

class Tab:
    def __init__(self):

        self.file_path = None
        self.data = None # Liste des dicts -> Les dicts sont les lignes du tableau
        self.columns = None # Liste des colonnes
        self.columns_type = None


    def copy(self):
        tab_copy = Tab()
        tab_copy.data = copy.deepcopy(self.data)
        tab_copy.columns = copy.deepcopy(self.columns)
        tab_copy.columns_type = copy.deepcopy(self.columns_type)
        return tab_copy


    def size(self):
        return len(self.data)

    
    # def print_file_path(self):
    #     print(self.file_path)
        

    # def print_columns_type(self):
    #     print(self.columns_type)

    
    def show(self):

        data_dict = self.data 
        columns = self.columns
        
        cell_width = 12
        
        header = "|".join(column.center(cell_width) for column in columns)
        
        print(header)
        print("-" * len(header))
        
        for line in data_dict:
            
            list_cell_values = []
            for column in columns:
                cell_value = str(line[column])
                                 
                if(len(cell_value)>cell_width):
                    cell_value = cell_value[:cell_width-3]+"..."
    
                cell_value = cell_value.ljust(cell_width)
                list_cell_values.append(cell_value)
                    
            row ="|".join(list_cell_values)
            
            print(row)


    
    
    def save(self,filename,path,file_type):
        filename=f"{filename}.{file_type}"
        
        if file_type == 'csv':
            save_csv(self.data,path,filename)

        elif file_type == 'json':
            save_json(self.data,path,filename)

    
    def load(self,file_path): # Load un nouveau Tab depuis in fichier
        self.file_path = file_path

        file_type = get_file_type(file_path)
        
        if(file_type == 'csv'):
            data = load_csv(file_path)

        elif file_type == 'json':
            data = load_json(file_path)

            
        else:
            print(f"{file_type} is'nt supported")

        
        self.columns_type = get_column_types(data)
        self.columns = list(self.columns_type.keys())
                    
        self.data = data
                


    
    # def add_line(self, **kwargs):
    #     if not all(key in self.columns for key in kwargs.keys()):
    #         print("ERROR")
    #     else:
    #         line_dict = {column:None for column in self.columns}
    #         for key, value in kwargs.items():
    #             line_dict[key] = value

    #         self.data.append(line_dict)
    #         self.size += 1
            
    #     return self
        

    
    def add_columns(self,*new_columns):

        new_columns = list(set(new_columns)-set(self.columns))
        
        self.columns += new_columns
        
        for i in range (0,self.size()):
            
            for column in new_columns:
                self.data[i][column] = None
                
        return self
        

    def remove_columns(self,*columns):

        columns = list(set(columns)&set(self.columns))

        for column in columns:
            self.columns.remove(column)

        
        for i in range (0,self.size()):
            for column in columns:
                self.data[i].pop(column)
        return self

    
    def convert_column_type(self, column, type):
        
        if(type=="int"):
            self.columns_type[column]={"int"}
            for i,row in enumerate(self.data):
                try:
                    self.data[i][column] = int(row[column])
                except Exception:
                    self.data[i][column] = None
                    
        elif(type=="float"):
            self.columns_type[column]={"float"}
            for i,row in enumerate(self.data):
                try:
                    self.data[i][column] = float(row[column])
                except Exception:
                    self.data[i][column] = None
                    
        if(type=="str"):
            self.columns_type[column]={"str"}
            for i,row in enumerate(self.data):
                try:
                    self.data[i][column] = str(row[column])
                except Exception:
                    self.data[i][column] = None
        return self
    
        
    def statistics(self):
        stats = {column:{} for column in self.columns}
        sample = self.data[0]

        for column,type in self.columns_type.items():
            value = sample[column]
            if type == "int" or type == "float":
                stats[column]["Min"]= value
                stats[column]["Max"]= value
                stats[column]["Mean"]= value

            if type == "str":
                stats[column]["Min"]= value
                stats[column]["Max"]= value
                stats[column]["Most Frequent"]= [value]
                
        count = 0
        
        for row in self.data:
            count += 1
            for column,type in self.columns_type.items():
                value = row[column]
                
                try:
                    if type == "int" or type == "float":
                        
                        if(value < stats[column]["Min"]):
                            stats[column]["Min"]= value    
    
                        if(value > stats[column]["Max"]):
                            stats[column]["Max"]= value
    
                        stats[column]["Mean"] += value
                        
        
                    if type == "str" and value!=None:
                        
                        if(len(value) < len(stats[column]["Min"])):
                            stats[column]["Min"]= value    
    
                        if(len(value) > len(stats[column]["Max"])):
                            stats[column]["Max"]= value
                        
                        stats[column]["Most Frequent"].append(value)

                except TypeError:
                    pass
                    
        for column,type in self.columns_type.items():
            if type == "int" or type == "float":
                stats[column]["Mean"] = round(stats[column]["Mean"]/count,2)
                
            if type == "str":
                list = stats[column]["Most Frequent"]
                stats[column]["Most Frequent"] = max(set(list),key=list.count)

        print(json.dumps(stats, indent=4))


    
    def sort(self,column,reverse=False):
        infini = float('inf')
        if reverse:
            infini = float('-inf')

        # self.data = sorted(self.data,key=lambda x: sum_ord(str(x[column])) if x[column] is not None else infini, reverse = reverse)
        
        if(str in self.columns_type[column]):
            self.data = sorted(self.data,key=lambda x: sum_ord(str(x[column])) if x[column] is not None else infini, reverse = reverse)
            
        else:
            self.data = sorted(self.data,key=lambda x: x[column] if x[column] is not None else infini, reverse = reverse)            
        return self

    

    def filter(self,column,rel,value):
        
        sub_tab = Tab()
        
        sub_tab.columns = self.columns
        sub_tab.columns_type = self.columns_type
        sub_tab.data = []

        
        if(int in self.columns_type[column] or float in self.columns_type[column]):
            if(rel=="IS EQUAL"):
                for row in self.data:
                    # print(row[column])
                    if(row[column] == value):
                        sub_tab.data.append(row)
    
            if(rel=="IS GREATER THAN"):
                
                for row in self.data:
                    try:
                        if(row[column] > value):
                            sub_tab.data.append(row)
                    except TypeError:
                        pass
    
            if(rel=="IS GREATER THAN OR EQUAL"):
                for row in self.data:
                    try:
                        if(row[column] >= value):
                            sub_tab.data.append(row)
                    except TypeError:
                        pass
    
            if(rel=="IS LESS THAN"):
                for row in self.data:
                    try:
                        if(row[column] < value):
                            sub_tab.data.append(row)
                    except TypeError:
                        pass
                        
            if(rel=="IS LESS THAN OR EQUAL"):
                for row in self.data:
                    try:
                        if(row[column] <= value):
                            sub_tab.data.append(row)
                    except TypeError:
                        pass

        # Si la colonne est string
        elif(str in self.columns_type[column]):
            if(rel=="IS EQUAL"):
                
                for row in self.data:
                    if(row[column] == value):
                        sub_tab.data.append(row)
    
            if(rel=="IS GREATER THAN"):
                for row in self.data:
                    try:
                        if(sum_ord(row[column]) > value):
                            sub_tab.data.append(row)
                    except TypeError:
                        pass
    
            if(rel=="IS GREATER OR EQUAL THAN"):
                for row in self.data:
                    try:
                        if(sum_ord(row[column]) >= value):
                            sub_tab.data.append(row)
                    except TypeError:
                        pass
    
            if(rel=="IS LESS THAN"):
                for row in self.data:
                    try:
                        if(sum_ord(row[column]) < value):
                            sub_tab.data.append(row)
                    except TypeError:
                        pass
                        
            if(rel=="IS LESS OR EQUAL THAN"):
                for row in self.data:
                    try:
                        if(sum_ord(row[column]) <= value):
                            sub_tab.data.append(row)
                    except TypeError:
                        pass       
                    
        return sub_tab

0
PassengerId |   Pclass   |    Name    |    Sex     |    Age     |   SibSp    |   Parch    |   Ticket   |    Fare    |   Cabin    |  Embarked  
----------------------------------------------------------------------------------------------------------------------------------------------
892         |3           |Kelly, Mr...|male        |34.5        |0           |0           |330911      |7.8292      |            |Q           
893         |3           |Wilkes, M...|female      |47          |1           |0           |363272      |7           |            |S           
894         |2           |Myles, Mr...|male        |62          |0           |0           |240276      |9.6875      |            |Q           
895         |3           |Wirz, Mr....|male        |27          |0           |0           |315154      |8.6625      |            |S           
896         |3           |Hirvonen,...|female      |22          |1           |1           |3101298     |12.2875     |            |S         

In [44]:
new_tab = Tab()
new_tab.load("../data/csv/titanic.csv")
new_tab.convert_column_type(column="Age",type="float").show()

PassengerId |   Pclass   |    Name    |    Sex     |    Age     |   SibSp    |   Parch    |   Ticket   |    Fare    |   Cabin    |  Embarked  
----------------------------------------------------------------------------------------------------------------------------------------------
892         |3           |Kelly, Mr...|male        |34.5        |0           |0           |330911      |7.8292      |            |Q           
893         |3           |Wilkes, M...|female      |47.0        |1           |0           |363272      |7           |            |S           
894         |2           |Myles, Mr...|male        |62.0        |0           |0           |240276      |9.6875      |            |Q           
895         |3           |Wirz, Mr....|male        |27.0        |0           |0           |315154      |8.6625      |            |S           
896         |3           |Hirvonen,...|female      |22.0        |1           |1           |3101298     |12.2875     |            |S           

In [168]:
import tkinter as tk
from tkinter import ttk,simpledialog


# Fonction bouton load new table de la table
def load_table(c_tab=None):
    
    #New tab
    tab = Tab()
    
    if(c_tab is None):
        user_input = simpledialog.askstring("Load new table", "Enter your file path :")
        
        if user_input is not None:
            # Chargemant de Tab
            tab.load(user_input)
    else:
        tab = c_tab

    
    #tab copy
    original_tab = tab.copy()
    
    #Liste des modifications
    modifications = []
    curr_mod = 0
    
    root = tk.Tk()
    root.title(tab.file_path)


    # Treeview pour faire des tableaux
    tree = ttk.Treeview(root)
    
    def modification_changes():
        nonlocal curr_mod
        nonlocal modifications
        modifications = modifications[:curr_mod+1]
        modifications.append(tab.copy())
        curr_mod+=1
        
    modification_changes()
    
    def add_column():
        nonlocal tab
        nonlocal curr_mod
        nonlocal modifications

        def apply_add():
            nonlocal tab
            nonlocal curr_mod
            nonlocal modifications

            column = variable_column.get()
            
            for item in tree.get_children():
                tree.delete(item)
            
            tab = tab.add_columns(column)
            # tab.show()
            
            tree["columns"] = tab.columns
            
            # Mettez à jour votre tableau avec les nouvelles dimensions
            tree.update_idletasks()

            for col in columns:
                tree.column(col, anchor="center")
                tree.heading(col, text=col, anchor="center")
            
            for item in tab.data:
                values = [item[col] for col in tab.columns]
                tree.insert("", "end", values=values)
                
            # modification_changes()
            # modifications[-1].show()

            root.destroy()
            
        
        root = tk.Tk()
        root.title("Column")

        # variables
        variable_column = ttk.Entry(root)
        variable_column.grid(row=2, column=1, padx=10,pady=5)

        # Bouton add
        button_sort = ttk.Button(root, text="Add", command=apply_add)
        button_sort.grid(row=3, column=0, columnspan=2, pady=10)    


    
    def remove_column():
        nonlocal tab
        nonlocal curr_mod
        nonlocal modifications

        def apply_remove():
            nonlocal tab
            nonlocal curr_mod
            nonlocal modifications

            column = variable_column.get()
            
            for item in tree.get_children():
                tree.delete(item)
            
            tab = tab.remove_columns(column)

            tree.column(column, width=0, stretch=tk.NO)
            tree.heading(column, text="")
            
            # Mettez à jour votre tableau avec les nouvelles dimensions
            tree.update_idletasks()
            
            for item in tab.data:
                values = [item[col] for col in tab.columns]
                tree.insert("", "end", values=values)
                
            modification_changes()
            root.destroy()
            
    
        root = tk.Tk()
        root.title("Remove column")

        # variables
        variable_column = tk.StringVar(root)
        
        # Menu déroulant colonnes
        label_column = ttk.Label(root, text="Column")
        tmp = [""]+tab.columns
        menu_column = ttk.OptionMenu(root, variable_column, *tmp)
    
        label_column.grid(row=0, column=0, padx=10, pady=5, sticky="e")
        menu_column.grid(row=0, column=1, padx=10, pady=5)
        # user_input = simpledialog.askstring("Sort table", "On :")
    
        # Bouton remove
        button_sort = ttk.Button(root, text="Remove", command=apply_remove)
        button_sort.grid(row=3, column=0, columnspan=2, pady=10)
            
        
    
    def reset():
        nonlocal tab
        nonlocal curr_mod
        nonlocal modifications
        
        for item in tree.get_children():
            tree.delete(item)

        for item in original_tab.data:
            values = [item[col] for col in columns]
            tree.insert("", "end", values=values)
            
        tab = curr_mod[0]
        modification_changes()

        
    def sort_table():
        nonlocal tab
        nonlocal curr_mod
        nonlocal modifications
        # nonlocal modifications
        
        def apply_sort():
            # modifications[0].show()
            nonlocal tab
            nonlocal curr_mod
            nonlocal modifications
            # Effacer le tableau (tree)
            for item in tree.get_children():
                tree.delete(item)

            column = variable_column.get()
            reverse = variable_reverse.get() == "Yes" 
            sorted_tab = tab.sort(column=column,reverse=reverse)
            
            for item in sorted_tab.data:
                values = [item[col] for col in columns]
                tree.insert("", "end", values=values)

            tab = sorted_tab

            # Modifications sauvegarde
            modification_changes()



        root = tk.Tk()
        root.title("Sort")
        
        # variables
        variable_column = tk.StringVar(root)
        variable_reverse = tk.StringVar(root)
        
        # Menu déroulant colonnes
        label_column = ttk.Label(root, text="Column")
        tmp = [""]+tab.columns
        menu_column = ttk.OptionMenu(root, variable_column, *tmp)

        # Menu déroulant colonnes
        label_reverse = ttk.Label(root, text="Reverse")
        tmp_reverse = ["","No","Yes"]
        menu_column_reverse = ttk.OptionMenu(root, variable_reverse, *tmp_reverse)
    
        label_column.grid(row=0, column=0, padx=10, pady=5, sticky="e")
        menu_column.grid(row=0, column=1, padx=10, pady=5)

        label_reverse.grid(row=1, column=0, padx=10, pady=5, sticky="e")
        menu_column_reverse.grid(row=1, column=1, padx=10, pady=5)
    
        # Bouton sort
        button_sort = ttk.Button(root, text="Sort", command=apply_sort)
        button_sort.grid(row=3, column=0, columnspan=2, pady=10)

        

    
    # Fonction bouton filter de la table
    def filter_table():
        nonlocal tab         
        nonlocal modifications
        nonlocal curr_mod
        
        
        def apply_filter():
            nonlocal tab
            nonlocal modifications
            nonlocal curr_mod
            
            for item in tree.get_children():
                tree.delete(item)
            
            column = variable_column.get()
            rel = variable_filter.get()
            value = variable_value.get()      


            if(rel=="IS EQUAL" and str in tab.columns_type[column]):
                print("HERE")
                filter_tab = tab.filter(column=column,rel=rel,value=value)
            else:
                filter_tab = tab.filter(column=column,rel=rel,value=int(value))
            # root.destroy()
            # load_table(c_tab = filter_tab)
            for item in filter_tab.data:
                values = [item[col] for col in columns]
                tree.insert("", "end", values=values)
                
            tab = filter_tab

            
            # Modifications sauvegarde
            modification_changes()

                
        root = tk.Tk()
        root.title("Filter")
        
        
        # variables
        variable_column = tk.StringVar(root)
        variable_filter = tk.StringVar(root)
        variable_value = tk.StringVar(root)
        
    
        # Menu déroulant colonnes
        label_column = ttk.Label(root, text="Column")
        tmp = [""]+tab.columns
        menu_column = ttk.OptionMenu(root, variable_column, *tmp)
    
    
        label_column.grid(row=0, column=0, padx=10, pady=5, sticky="e")
        menu_column.grid(row=0, column=1, padx=10, pady=5)
        
        # Menu déroulant filtres
        filters = ["","IS EQUAL","IS GREATER THAN","IS GREATER THAN OR EQUAL","IS LESS THAN","IS LESS THAN OR EQUAL"]
        
        label_filter = ttk.Label(root, text="Filter:")
        menu_filter = ttk.OptionMenu(root, variable_filter, *filters)
        label_filter.grid(row=1, column=0, padx=10, pady=5, sticky="e")
        menu_filter.grid(row=1, column=1, padx=10, pady=5)
        
        # value
        label_value = ttk.Label(root, text="Value:")
        variable_value = ttk.Entry(root)
        label_value.grid(row=2, column=0, padx=10, pady=5, sticky="e")
        variable_value.grid(row=2, column=1, padx=10, pady=5)
        
        # Bouton filter
        button_filter = ttk.Button(root, text="Filter", command=apply_filter)
        button_filter.grid(row=3, column=0, columnspan=2, pady=10)

            
    def convert_type():
        nonlocal tab
        nonlocal curr_mod
        nonlocal modifications
        # nonlocal modifications
        
        def apply_convert():
            # modifications[0].show()
            nonlocal tab
            nonlocal curr_mod
            nonlocal modifications
            # Effacer le tableau (tree)
            for item in tree.get_children():
                tree.delete(item)

            column = variable_column.get()
            type = variable_type.get()
            
            convert_tab = tab.convert_column_type(column=column,type=type)
            
            for item in convert_tab.data:
                values = [item[col] for col in columns]
                tree.insert("", "end", values=values)

            tab = convert_tab

            # Modifications sauvegarde
            modification_changes()



        root = tk.Tk()
        root.title("Sort")
        
        # variables
        variable_column = tk.StringVar(root)
        variable_type = tk.StringVar(root)
        
        # Menu déroulant colonnes
        label_column = ttk.Label(root, text="Column")
        tmp = [""]+tab.columns
        menu_column = ttk.OptionMenu(root, variable_column, *tmp)

        # Menu déroulant colonnes
        label_type = ttk.Label(root, text="Type")
        tmp_type = ["","int","float","str"]
        menu_column_type = ttk.OptionMenu(root, variable_type, *tmp_type)
    
        label_column.grid(row=0, column=0, padx=10, pady=5, sticky="e")
        menu_column.grid(row=0, column=1, padx=10, pady=5)

        label_type.grid(row=1, column=0, padx=10, pady=5, sticky="e")
        menu_column_type.grid(row=1, column=1, padx=10, pady=5)
    
        # Bouton convert
        button_sort = ttk.Button(root, text="Convert", command=apply_convert)
        button_sort.grid(row=3, column=0, columnspan=2, pady=10)

    
    def undo_redo(incr):
        nonlocal tab
        nonlocal modifications
        nonlocal curr_mod
        
        
        if(0<=curr_mod+incr<len(modifications)):
            
            for item in tree.get_children():
                tree.delete(item)

            curr_mod += incr

            to_delete_col = [col for col in tab.columns if col not in modifications[curr_mod].columns]
            
            for column in to_delete_col:
                tab = tab.remove_columns(column)
                tree.column(column, width=0, stretch=tk.NO)
                tree.heading(column, text="")
                
                # Mettez à jour votre tableau avec les nouvelles dimensions
                tree.update_idletasks()


            
            tab = modifications[curr_mod]

            print(curr_mod)
            print(modifications[curr_mod].columns)
            
            tree["columns"] = tab.columns
            
            # Mettez à jour votre tableau avec les nouvelles dimensions
            tree.update_idletasks()

            for col in columns:
                tree.column(col, anchor="center")
                tree.heading(col, text=col, anchor="center")

            
            for item in tab.data:
                values = [item[col] for col in columns]
                tree.insert("", "end", values=values) 
    
    
    # Fonction bouton sauvegarde de la table
    def save_table():
        nonlocal tab
        
        def save():
            nonlocal tab
            
            path = entry_path.get()
            filename = entry_filename.get()
            file_type = variable_type.get()
        
            if path and filename and file_type:
                result_string = f"{path}/{filename}.{file_type}"
                print(result_string)
                tab.save(filename,path,file_type)
                root.destroy()
            else:
                print("Veuillez remplir tous les champs.")
        
        # Creation fenetre
        root = tk.Tk()
        root.title("Save table")
        
        # les champs d entrées
        label_path = ttk.Label(root, text="Path:")
        entry_path = ttk.Entry(root)
        
        label_filename = ttk.Label(root, text="File name:")
        entry_filename = ttk.Entry(root)
        
        # label_type = ttk.Label(root, text="Type:")
        # entry_type = ttk.Entry(root)
    
        # Menu déroulant type
        type = ["","csv","json"]
        
        variable_type = tk.StringVar(root)
        label_type = ttk.Label(root, text="Type:")
        menu_type = ttk.OptionMenu(root, variable_type, *type)
        
        
        # Bouton save
        button_save = ttk.Button(root, text="Save", command=lambda : save(tab))
        
        # Grille de bouton
        label_path.grid(row=0, column=0, padx=10, pady=5, sticky="e")
        entry_path.grid(row=0, column=1, padx=10, pady=5)
        
        label_filename.grid(row=1, column=0, padx=10, pady=5, sticky="e")
        entry_filename.grid(row=1, column=1, padx=10, pady=5)
        
        label_type.grid(row=2, column=0, padx=10, pady=5, sticky="e")
        menu_type.grid(row=2, column=1, padx=10, pady=5)
        
        button_save.grid(row=3, column=0, columnspan=2, pady=10)

    
    button_load = ttk.Button(root, text="Load new table", command=load_table)
    button_add_column = ttk.Button(root, text="Add column", command=add_column)
    button_remove_column = ttk.Button(root, text="Remove column", command=remove_column)
    button_filter = ttk.Button(root, text="Filter", command=lambda : filter_table())
    button_save = ttk.Button(root, text="Save table", command=lambda: save_table())
    button_sort_table = ttk.Button(root, text="Sort Table", command=lambda: sort_table())
    button_convert_type = ttk.Button(root, text="Convert Type", command=lambda: convert_type())
    button_reset = ttk.Button(root, text="Reset", command=lambda : reset())
    button_undo = ttk.Button(root, text="Undo", command=lambda : undo_redo(-1))
    button_redo = ttk.Button(root, text="Redo", command=lambda : undo_redo(1))
    

    button_load.grid(row=0, column=1)
    button_save.grid(row=0, column=2)
    button_add_column.grid(row=0, column=3)
    button_remove_column.grid(row=0, column=4)
    button_filter.grid(row=0, column=5)
    button_sort_table.grid(row=0, column = 6)
    button_convert_type.grid(row=0,column=7)
    button_reset.grid(row=0,column=8)
    button_undo.grid(row=0,column=9)
    button_redo.grid(row=0,column=10)
    
    columns = tab.columns
    
    def tree_config():
        nonlocal tree
        nonlocal columns
        # scroll barre
        horizontal_scrollbar = ttk.Scrollbar(root, orient="horizontal", command=tree.xview)
        tree.configure(xscrollcommand=horizontal_scrollbar.set)
            
            
        # les colonnes
        
        tree["columns"] = columns
            
        for col in columns:
            tree.column(col, anchor="center")
            tree.heading(col, text=col, anchor="center")
            
        # les lignes
        for item in tab.data:
            values = [item[col] for col in columns]
            tree.insert("", "end", values=values)
            
        # treeview config
        tree.grid(row=1, column=0, columnspan=15, padx=10, pady=10, sticky="nsew")
        horizontal_scrollbar.grid(row=2, column=0, columnspan=4, sticky="ew")
        
        
        # truc de redimenssionnement
        root.columnconfigure(0, weight=1)
        root.rowconfigure(1, weight=1)
        
    tree_config()
    
    root.mainloop()


# lancement de l'interface
def launch():
    root = tk.Tk()
    root.geometry("400x400")
    root.title("Data Filter")
    root.resizable(height=True,width=True)
    
    title_label = ttk.Label(root, text="Data Filter", font=("Helvetica", 16, "bold"))
    title_label.pack(pady=5)
    
    button = ttk.Button(root, text="Load table", command=load_table)
    button.pack(padx=10, pady=10)
    
    root.mainloop()

In [169]:
def test():
    new_tab = Tab()
    new_tab.load("../data/csv/titanic.csv")
    load_table(new_tab)
test()

10
['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
9
['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
8
['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
7
['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
6
['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
5
['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
4
['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
3
['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
2
['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
1
['PassengerId', 'Pclass', 'Name', 